<a href="https://colab.research.google.com/github/nvaikunt/PromptBasedReranking/blob/main/BaselineTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/nvaikunt/PromptBasedReranking.git

Cloning into 'PromptBasedReranking'...
remote: Enumerating objects: 63, done.
remote: Counting objects: 100% (63/63), done.
remote: Compressing objects: 100% (60/60), done.
remote: Total 63 (delta 33), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (63/63), done.


In [2]:
!pip install -q condacolab
import condacolab
condacolab.install()

⏬ Downloading https://github.com/jaimergp/miniforge/releases/latest/download/Mambaforge-colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:42
🔁 Restarting kernel...


In [1]:
!conda --version

conda 4.14.0


In [10]:
%cd PromptBasedReranking/

/content/PromptBasedReranking


In [3]:
!bash setup.sh

Solving environment: | failed with initial frozen solve. Retrying with flexible solve.
Solving environment: - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | 

In [11]:
import torch
torch.cuda.is_available()

True

In [5]:
!python Baseline/download_dpr_data.py --resource data.wikipedia-split.psgs_w100

Requested resource from %s https://www.dropbox.com/s/bezryc9win2bha1/psgs_w100.tar.gz
Download root_dir %s ./
File to be downloaded as %s /content/PromptBasedReranking/downloads/data/wikipedia-split/psgs_w100.tsv

Downloaded to %s /content/PromptBasedReranking/downloads/data/wikipedia-split/psgs_w100.tar.gz
Uncompressing %s /content/PromptBasedReranking/downloads/data/wikipedia-split/psgs_w100.tar.gz
 Saved to %s ./downloads/data/wikipedia-split


In [6]:
!python Baseline/download_dpr_data.py --resource data.retriever-outputs.dpr.nq-train

Requested resource from %s https://www.dropbox.com/s/6g4erof4ifg8xea/nq-train.tar.gz
Download root_dir %s ./
File to be downloaded as %s /content/PromptBasedReranking/downloads/data/retriever-outputs/dpr/nq-train.json

Downloaded to %s /content/PromptBasedReranking/downloads/data/retriever-outputs/dpr/nq-train.tar.gz
Uncompressing %s /content/PromptBasedReranking/downloads/data/retriever-outputs/dpr/nq-train.tar.gz
 Saved to %s ./downloads/data/retriever-outputs/dpr


In [13]:

import torch
from torch.utils.data import Dataset, DataLoader
import transformers
import tokenizers
import datasets 
import pandas

wikipedia_txt = datasets.load_dataset("csv", data_files='downloads/data/wikipedia-split/psgs_w100.tsv', delimiter='\t')


Using custom data configuration default-cb3c6fef8f5e82e0
Found cached dataset csv (/root/.cache/huggingface/datasets/csv/default-cb3c6fef8f5e82e0/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a)


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
wikipedia_txt

In [12]:
nq_open = datasets.load_dataset("json",data_files="downloads/data/retriever-outputs/dpr/nq-train.json")

Using custom data configuration default-a7ea593b34a6dc21
Found cached dataset json (/root/.cache/huggingface/datasets/json/default-a7ea593b34a6dc21/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab)


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
nq_open["train"][2]["question"]

In [14]:
def get_top_k_pos(row, k, txt_database):
  ctxs = row["ctxs"]
  top_k = []
  for ctx in ctxs:
    if ctx["has_answer"]:
      text = txt_database[ctx["id"] - 1]["text"]
      top_k.append((text, "true"))
    if len(top_k) == k:
      break
  if len(top_k) == 0: 
    return []
  while len(top_k) < k:
    top_k.extend(top_k[:(k - len(top_k))])

  return top_k[:k]
  
def get_top_k_pos_neg(row, k, txt_database):
  ctxs = row["ctxs"]
  top_k_pos = []
  top_k_neg = []
  for ctx in ctxs:
    if ctx["has_answer"] and len(top_k_pos) < k:
      text = txt_database[ctx["id"] - 1]["text"]
      top_k_pos.append((text, "true"))
    if not ctx["has_answer"] and len(top_k_neg) < k: 
      text = txt_database[ctx["id"] - 1]["text"]
      top_k_neg.append((text, "false"))
    if len(top_k_pos) == k and len(top_k_neg):
      break 
  if len(top_k_pos) == 0: 
    return []
  while len(top_k_pos) < k:
    top_k_pos.extend(top_k_pos[:(k - len(top_k_pos))])
  while len(top_k_neg) < k:
    top_k_neg.extend(top_k_neg[:(k - len(top_k_neg))])
 
  return top_k_pos + top_k_neg



In [15]:
def create_pos_txt_col(example, k, txt_database):
  return {"pos_text": get_top_k_pos(example, k, txt_database)}

def create_pos_neg_txt_col(example, k, txt_database):
  return {"pos_neg_text": get_top_k_pos_neg(example, k, txt_database)}


In [16]:
from functools import partial
nq_open["train"] = nq_open["train"].map(partial(create_pos_txt_col, k=20, txt_database=wikipedia_txt["train"]), num_proc=4)

Loading cached processed dataset at /root/.cache/huggingface/datasets/json/default-a7ea593b34a6dc21/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab/cache-8de3e792c7da3355.arrow


Loading cached processed dataset at /root/.cache/huggingface/datasets/json/default-a7ea593b34a6dc21/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab/cache-e9479766e3dfbfda.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/json/default-a7ea593b34a6dc21/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab/cache-4c401d4674da5ca8.arrow


Loading cached processed dataset at /root/.cache/huggingface/datasets/json/default-a7ea593b34a6dc21/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab/cache-76f806a699dc476d.arrow


In [17]:
nq_open["train"] = nq_open["train"].map(partial(create_pos_neg_txt_col, k=10, txt_database=wikipedia_txt["train"]), num_proc=4)

Loading cached processed dataset at /root/.cache/huggingface/datasets/json/default-a7ea593b34a6dc21/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab/cache-8f48c6a08d5d3d97.arrow


Loading cached processed dataset at /root/.cache/huggingface/datasets/json/default-a7ea593b34a6dc21/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab/cache-72c0db32a6c7dbf5.arrow


Loading cached processed dataset at /root/.cache/huggingface/datasets/json/default-a7ea593b34a6dc21/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab/cache-1cfb7182a88d7f2e.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/json/default-a7ea593b34a6dc21/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab/cache-d892b785abc1daa5.arrow


In [ ]:
nq_open["train"][0]["pos_text"]

In [7]:
from tqdm import tqdm
def create_ranking_loss_baseline_examples(dataset, n=None):
  if not n:
    n = len(dataset)
  with_answer = 0 
  inputs = []
  targets = []
  for i in tqdm(range(n)):
    texts = dataset[i]["pos_neg_text"]
    if not texts: continue 
    question = dataset[i]["question"]
    current_inputs = [f"Question: {question} Passage: {text[0]} Relevant: " for text in texts]
    current_targets = [text[1] for text in texts]
    inputs.extend(current_inputs)
    targets.extend(current_targets)
    with_answer += 1 
  k = [len(targets)/(with_answer * 2)] * len(targets)
  return {"inputs": inputs, "targets": targets, "k_pos_neg": k}

def create_q_gen_baseline_examples(dataset, n=None):
  if not n:
    n = len(dataset)
  inputs = []
  targets = []
  with_answer = 0 
  for i in tqdm(range(n)):
    texts = dataset[i]["pos_text"]
    if not texts: continue 
    question = dataset[i]["question"]
    current_inputs = [f"Passage: {text[0]} Please write a question based on this passage" for text in texts]
    current_targets = [question for text in texts]
    inputs.extend(current_inputs)
    targets.extend(current_targets)
    with_answer += 1     
  k = [len(targets)/(with_answer)] * len(targets)
  return {"inputs": inputs, "targets": targets, "k_pos": k}

def create_q_gen_ranking_baseline_examples(dataset, n=None):
  if not n:
    n = len(dataset)
  inputs = []
  targets = []
  with_answer = 0 
  for i in tqdm(range(n)):
    texts = dataset[i]["pos_neg_text"]
    if not texts: continue 
    question = dataset[i]["question"]
    current_inputs = [f"Passage: {text[0]} Please write a question based on this passage" for text in texts]
    current_targets = [question for text in texts]
    inputs.extend(current_inputs)
    targets.extend(current_targets)
    with_answer += 1   
  k = [len(targets)/(with_answer * 2)] * len(targets)
  return {"inputs": inputs, "targets": targets, "k_pos_neg": k}



In [ ]:
training_dict = create_q_gen_baseline_examples(nq_open["train"])

In [ ]:
q_gen_dataset = datasets.Dataset.from_dict(training_dict)

In [2]:
from transformers import AutoTokenizer
from transformers import T5ForConditionalGeneration

model_checkpoint = "google/t5-base-lm-adapt"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

In [ ]:
q_gen_dataset[2000]

In [97]:
def preprocess_function(examples, max_input_length, max_target_length, input_col):
    model_inputs = tokenizer(
        examples[input_col],
        max_length=max_input_length,
        truncation=True,  padding="longest"
    )
    labels = tokenizer(text_target=examples["targets"], max_length=max_target_length, truncation=True,  padding="longest", return_tensors="pt"
    )
    model_inputs["labels"] = labels["input_ids"]
    model_inputs["labels_mask"] = labels["attention_mask"]
    return model_inputs

In [4]:
from transformers import T5ForConditionalGeneration
model = T5ForConditionalGeneration.from_pretrained(model_checkpoint)

In [46]:
import torch
import datasets
eval_subset = datasets.Dataset.from_dict(create_q_gen_baseline_examples(nq_open["train"].select(range(4))))
eval_dataset = eval_subset.map(partial(preprocess_function, max_input_length=300, max_target_length=50, input_col='inputs'), batched=True)
eval_dataset.set_format(type="torch", columns=["input_ids", "labels", "attention_mask", "labels_mask"])


100%|██████████| 4/4 [00:00<00:00, 82.90it/s]


  0%|          | 0/1 [00:00<?, ?ba/s]

In [68]:
nq_open["train"]

Dataset({
    features: ['question', 'answers', 'ctxs', 'pos_text', 'pos_neg_text'],
    num_rows: 79168
})

In [102]:
i = 0
k = 100
txt_database = wikipedia_txt["train"]
question = nq_open["train"][i]["question"]
ctxs = nq_open["train"][i]["ctxs"][:k]
has_ans = [ctx["has_answer"] for ctx in ctxs]
texts = [txt_database[ctx["id"] - 1]["text"] for ctx in ctxs]
texts = [f"Passage: {text} Please write a question based on this passage" for text in texts]
targets = [question for text in texts]
new_dataset = datasets.Dataset.from_dict({'inputs': texts, 'targets': targets})
new_dataset = new_dataset.map(partial(preprocess_function, max_input_length=300, max_target_length=50, input_col='inputs'), batched=True)
new_dataset.set_format(type="torch", columns=["input_ids", "labels", "attention_mask", "labels_mask"])
new_dataset["input_ids"][0:3]


  0%|          | 0/1 [00:00<?, ?ba/s]

tensor([[ 3424,   545,    10,  4366,    23,   172,   795,   596,   747,  2279,
             5,    37,   372,  2301,    12,   549,   683,   448,    21,     8,
          1421, 10439,   774,     6,  7784,     3,     9,   460,    18,  1201,
          1675,    28, 19856,  5061,    18,  9160,   549,     4,   476,   382,
            18, 14908,     5,    37,  1357,    12,   294,    28,   549,     4,
           476,   382,    47,     3, 15733,    16,     7,  2880,   920,    57,
             3,     9,  2173,    57,     8,   372,    21,     8,  2478,    12,
          1472,    30,    18,  2256,  6794,  4794,  3833,   295,    23,   318,
          8903,    65,   141,     3,     9,   892,    13,   492,  2622,  2404,
            13,     8, 10371,     7,   383,   112,  1262,    18,   715,   504,
           318,     9,     7,     3,     9,  1706,    13,   136,   647, 16868,
             5,    71, 19856,  5061, 23909,  4568,    24,    70, 27920,    47,
          3679,    12,  1961,     8,  2478,    31,  

In [57]:
#eval_dataset = eval_dataset.remove_columns(['inputs', 'targets', 'k_pos'])
eval_inputs = eval_dataset["input_ids"][0:20]
eval_masks = eval_dataset["attention_mask"][0:20]
target_eval_labels = eval_dataset["labels"][0:20]
target_eval_label_mask= eval_dataset["labels_mask"][0:20]
decoder_input_ids = model.prepare_decoder_input_ids_from_labels(labels=eval_dataset["labels"][0:20])

In [59]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
eval_masks.to(device)
eval_inputs.to(device)
target_eval_labels.to(device)
target_eval_label_mask.to(device)
outputs = model(input_ids=eval_inputs, labels = target_eval_labels, attention_mask=eval_masks, decoder_attention_mask=target_eval_label_mask, decoder_input_ids=decoder_input_ids)

In [70]:
decoder_input_ids

tensor([[  0, 116,  47,  ...,   0,   0,   0],
        [  0, 116,  47,  ...,   0,   0,   0],
        [  0, 116,  47,  ...,   0,   0,   0],
        ...,
        [  0, 113,  31,  ...,   0,   0,   0],
        [  0, 113,  31,  ...,   0,   0,   0],
        [  0, 113,  31,  ...,   0,   0,   0]])

In [63]:
import torch.nn 
logits = outputs.logits
log_softmax = torch.nn.LogSoftmax(dim=-1)
log_soft = log_softmax(logits)

In [64]:
labels = target_eval_labels.unsqueeze(2)
log_soft = log_soft.gather(2, labels).squeeze(2)
log_soft = log_soft.mean(dim=1)
topk_scores, indexes = torch.topk(log_soft, k=len(log_soft))
topk_scores


tensor([-29.4132, -29.8577, -30.4236, -30.6911, -31.0089, -31.3089, -31.4755,
        -31.7324, -31.9018, -32.0330, -32.1019, -32.2398, -32.2956, -32.5171,
        -32.6039, -32.7928, -32.8235, -32.8541, -32.9538, -33.5006],
       grad_fn=<TopkBackward0>)

In [66]:
-topk_scores.mean()
outputs.loss


tensor(31.8265, grad_fn=<NllLossBackward0>)

In [ ]:
def evaluate_questions(validation, k, model, tokenizer, batch_size, dataset_generator=None):
  if not dataset_generator:
    dataset_generator = create_q_gen_baseline_examples
  assert(k // batch_size != 0, "k must be multiple of batch_size")
  for i in range(len(validation)):
    question = validation[i]["question"]
    ctxs = validation[i]["ctxs"][:k]
    has_ans = [ctx["has_answer"] for ctx in ctxs]
    for i in tqdm(range(0, k, batch_size)):



    



torch.Size([50, 50, 32128])

In [104]:
test_nq = nq_open["train"].select(range(5, 205))
test_nq = test_nq.map(partial(create_pos_txt_col, k=20, txt_database=wikipedia_txt["train"]), num_proc=4)
training_dict = create_q_gen_baseline_examples(test_nq)
test_train = datasets.Dataset.from_dict(training_dict)
test_train = test_train.map(partial(preprocess_function, max_input_length=300, max_target_length=50, input_col='inputs'), batched=True)


#0:   0%|          | 0/50 [00:00<?, ?ex/s]

#1:   0%|          | 0/50 [00:00<?, ?ex/s]

#3:   0%|          | 0/50 [00:00<?, ?ex/s]

#2:   0%|          | 0/50 [00:00<?, ?ex/s]

100%|██████████| 200/200 [00:02<00:00, 83.87it/s]


Dataset({
    features: ['inputs', 'targets', 'k_pos'],
    num_rows: 3660
})

In [ ]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments, Seq2SeqTrainer
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)
trainer_args = Seq2SeqTrainingArguments(
    output_dir=f"test_1",
    evaluation_strategy="epoch",
    learning_rate=5.6e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    num_train_epochs=3,
    predict_with_generate=True,
)
trainer =  Seq2SeqTrainer(
    model,
    trainer_args,
    train_dataset=tokenized_datasets["train"],
    data_collator=data_collator,
    tokenizer=tokenizer
)


100%|██████████| 50/50 [00:00<00:00, 89.89it/s]


In [ ]:
dataset_gen[49]

{'inputs': "Passage: in oceans and other large bodies of water, with 1.6% of water below ground in aquifers and 0.001% in the air as vapor, clouds, and precipitation. Oceans hold 97% of surface water, glaciers, and polar ice caps 2.4%, and other land surface water such as rivers, lakes, and ponds 0.6%. Additionally, a minute amount of the Earth's water is contained within biological bodies and manufactured products. An ocean is a major body of saline water, and a principal component of the hydrosphere. Approximately 71% of the Earth's surface (an area of some 361 million square kilometers) is covered by ocean, Please write a question based on this passage",
 'targets': 'which part of earth is covered with water',
 'k_pos': 20.0}